In [ ]:
!pip install fastparquet

In [ ]:
!pip install awswrangler

In [5]:
import boto3
import pandas as pd
import numpy as np
import re
from sagemaker import get_execution_role
import awswrangler as wr
import os, sys
import tqdm
role = get_execution_role()
s3 = boto3.resource('s3')

In [6]:
icusics_db_patients = wr.s3.read_parquet(path="s3://icusics-db/patients/patients.parquet")
scr= pd.read_csv("scr_all_hosp.csv")
uo = pd.read_csv("uo_all_hosp.csv")

CREATININE - KDIGO AKI STAGE 

In [15]:
cols = {'a_patientid':'a_patientid', 'time':'time_48', 'value':'value_48'}
cr48 = scr.loc[:, cols.keys()].rename(columns=cols)

In [16]:
m_48 = scr.merge(cr48, how='left', on=['a_patientid'])
m_48 = m_48.query("time_48 < time and time_48 > time - 2880")
m_48 = m_48.merge(scr, how='right', on=scr.columns.tolist())
m_48 = m_48.groupby(['a_patientid','a_variableid','time','value']).value_48.min().reset_index()

cols = {'a_patientid':'a_patientid', 'time':'time_7', 'value':'value_7'}
cr7 = scr.loc[:, cols.keys()].rename(columns=cols)

m_7 = scr.merge(cr7, how='left', on=['a_patientid'])
m_7 = m_7.query("time_7 < time and time_7 > time - 10080")
m_7 = m_7.merge(scr, how='right', on=scr.columns.tolist())
m_7 = m_7.groupby(['a_patientid','a_variableid','time','value']).value_7.min().reset_index()

kdigo_creatinine = m_48.merge(m_7, how='inner', on=['a_patientid', 'a_variableid', 'time', 'value'])

In [9]:
kdigo_creatinine

,a_patientid,a_variableid,time,value,value_48,value_7
0,1000091,1020000600,1216,0.86,NaN,NaN
1,1000091,1020000600,2685,0.75,0.86,0.86
2,1000091,1020000600,4185,0.65,0.75,0.75
3,1000109,1020000600,-6115,1.28,NaN,NaN
4,1000109,1020000600,-4870,1.43,1.28,1.28
...,...,...,...,...,...,...
126687,6998846,6020000600,988,0.75,0.82,0.82
126688,6999255,6020000600,134,0.60,NaN,NaN
126689,6999255,6020000600,1444,0.57,0.60,0.60
126690,6999255,6020000600,2874,0.56,0.57,0.57


DIURESIS - KDIGO AKI STAGE

In [10]:
def get_lag(df, l, merge_idx=['a_patientid'], agg_fnc=np.sum, agg_col='value'):
    orig_cols = df.columns.tolist()
    custom_cols = list(map(lambda x: x + '_' + str(l) if x not in merge_idx else x, orig_cols))

    cols = {'time': f'time_{l}', 'value': f'value_{l}'}
    df_l = df.rename(columns=cols)

    m_l = df.merge(df_l, how='left', on=merge_idx)
    lhr = l * 60
    q_str = f"{cols['time']} < time and {cols['time']} > time - {lhr}"
    m_l = m_l.query(q_str)
    m_l = m_l.merge(df, how='right', on=df.columns.tolist())
    m_l = m_l.groupby(df.columns.tolist())[cols[agg_col]].apply(agg_fnc).reset_index()
    return m_l

In [11]:
u_patients = uo.a_patientid.unique()

l6 = []
t6 = []
l12 = []
t12 = []
l24 = []
t24 = []

delta = 100
for i in tqdm.tqdm(range(0, u_patients.shape[0], delta)):
    p_round = u_patients[i:i+delta]
    df = uo[uo.a_patientid.isin(p_round)].loc[:, ['a_patientid', 'time', 'value']]
    i6 = get_lag(df, l=6)
    i12 = get_lag(df, l=12)
    i24 = get_lag(df, l=24)
    j6 = get_lag(df, l=6, agg_col='time', agg_fnc=np.min)
    j12 = get_lag(df, l=12, agg_col='time', agg_fnc=np.min)
    j24 = get_lag(df, l=24, agg_col='time', agg_fnc=np.min)
    l6.append(i6)
    l12.append(i12)
    l24.append(i24)
    t6.append(j6)
    t12.append(j12)
    t24.append(j24)

100%|██████████| 209/209 [18:21<00:00,  5.27s/it]


In [13]:
uo_l6 = pd.concat(l6, ignore_index=True)
uo_l12 = pd.concat(l12, ignore_index=True)
uo_l24 = pd.concat(l24, ignore_index=True)

uo_t6 = pd.concat(t6, ignore_index=True)
uo_t12 = pd.concat(t12, ignore_index=True)
uo_t24 = pd.concat(t24, ignore_index=True)

uo_t6['uo_tm_6hr'] = (uo_t6.time - uo_t6.time_6) / 60
uo_t12['uo_tm_12hr'] = (uo_t12.time - uo_t12.time_12) / 60
uo_t24['uo_tm_24hr'] = (uo_t24.time - uo_t24.time_24) / 60

uo_t6 = uo_t6.drop(columns='time_6')
uo_t12 = uo_t12.drop(columns='time_12')
uo_t24 = uo_t24.drop(columns='time_24')

In [14]:
uo_m1 = uo_l6.merge(uo_l12, on=['a_patientid','time','value'])
uo_m2 = uo_m1.merge(uo_l24, on=['a_patientid','time','value'])
uo_m3 = uo_m2.merge(uo_t6, on=['a_patientid','time','value'])
uo_m4 = uo_m3.merge(uo_t12, on=['a_patientid','time','value'])
uo_m5 = uo_m4.merge(uo_t24, on=['a_patientid','time','value'])

In [32]:
#uo_m5.to_csv("tmp.csv", index=True)

In [8]:
uo_m5=pd.read_csv("tmp.csv")

In [9]:
patients = icusics_db_patients[['a_patientid','weight']]

In [10]:
patients

,a_patientid,weight
0,1000091,50
1,1000109,80
2,1000211,60
3,1000999,60
4,1001000,80
...,...,...
21134,6998109,70
21135,6998382,100
21136,6998565,90
21137,6998846,80


In [11]:
uo_m6 = uo_m5.merge(patients, how='inner', on='a_patientid')


In [12]:
uo_m6['uo_rt_6hr'] = uo_m6.value_6 / uo_m6.weight / uo_m6.uo_tm_6hr
uo_m6['uo_rt_12hr'] = uo_m6.value_12 / uo_m6.weight / uo_m6.uo_tm_12hr
uo_m6['uo_rt_24hr'] = uo_m6.value_24 / uo_m6.weight / uo_m6.uo_tm_24hr

kdigo_uo = uo_m6.copy()

In [13]:
def creatine_stage(v, v48, v7):
    if (v >= v7*3) or ((v >= 4) and ((v48 <= 3.7) or (v >= 1.5*v7))):
        stage = 3
    elif (v >= v7*2):
        stage = 2
    elif (v >= v48+0.3) or (v >= 1.5*v7):
        stage = 1
    else:
        stage = 0
    return stage

In [17]:
aki_creat = list(map(lambda v,v48,v7: creatine_stage(v, v48, v7), kdigo_creatinine.value, kdigo_creatinine.value_48, kdigo_creatinine.value_7))
kdigo_creatinine['aki_stage_creat'] = aki_creat

In [18]:
cols = {'a_patientid':'a_patientid', 'time':'time', 
        'value':'creat', 'value_48':'creat_48hr', 'value_7':'creat_7days',
       'aki_stage_creat':'aki_stage_creat'}
kdigo_creatinine = kdigo_creatinine.loc[:, cols.keys()].rename(columns=cols)

In [19]:
def urineOut_stage(time, uo_tm_6hr, uo_tm_12hr, uo_tm_24hr, uo_rt_6hr, uo_rt_12hr, uo_rt_24hr):
    if (uo_rt_6hr  is None):
        stage = None
    elif time < 360:
        stage = 0
    elif ((uo_tm_24hr >= 11) and (uo_rt_24hr < 0.3)) or ((uo_tm_12hr >= 5) and (uo_rt_12hr == 0)):
        stage = 3
    elif (uo_tm_12hr >= 5) and (uo_rt_12hr < 0.5):
        stage = 2
    elif (uo_tm_6hr >= 2) and (uo_rt_6hr  < 0.5):
        stage = 1
    else:
        stage = 0
    return stage

In [ ]:
kdigo_uo['aki_stage_uo'] = list(map(lambda t,tm6,tm12,tm24,rt6,rt12,rt24: urineOut_stage(t,tm6,tm12,tm24,rt6,rt12,rt24), 
        uo_m6.time, 
        uo_m6.uo_tm_6hr,
        uo_m6.uo_tm_12hr,
        uo_m6.uo_tm_24hr,
        uo_m6.uo_rt_6hr,
        uo_m6.uo_rt_12hr,
        uo_m6.uo_rt_24hr))

In [ ]:
tm_stg = pd.concat(
    [kdigo_creatinine.loc[:, ['a_patientid','time']],
     kdigo_uo.loc[:, ['a_patientid','time']]
    ], ignore_index=True
)

In [ ]:
kdigo_stages = patients.loc[:, ['a_patientid']].merge(tm_stg, how='left', on='a_patientid')
kdigo_stages = kdigo_stages.merge(kdigo_creatinine, how='left', on=['a_patientid','time'])
uo_cols = ['a_patientid','time','uo_rt_6hr','uo_rt_12hr','uo_rt_24hr','aki_stage_uo']
kdigo_stages = kdigo_stages.merge(kdigo_uo.loc[:, uo_cols], how='left', on=['a_patientid','time'])

kdigo_stages['aki_stage'] = list(map(lambda x,y: max(x,y), kdigo_stages.aki_stage_creat.fillna(0).astype(float), kdigo_stages.aki_stage_uo.fillna(0).astype(float)))

kdigo_stages

In [ ]:
kdigo_stages.to_csv('kdigo_stages_all_hosp.csv')